<a href="https://colab.research.google.com/github/brauliomendez/prueba-sostenibilidad/blob/main/Prueba_t%C3%A9cnica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip SostenibilidadEmpresarial.zip

Archive:  SostenibilidadEmpresarial.zip
   creating: SostenibilidadEmpresarial/
  inflating: SostenibilidadEmpresarial/Empresas_Sostenibilidad.csv  
  inflating: SostenibilidadEmpresarial/Proyectos_Energeticos.csv  
  inflating: SostenibilidadEmpresarial/Regulaciones_Ambientales.csv  


Pyspark está instalado por defecto en Google Colab

In [2]:
# !pip install pyspark

### Dependencias necesarias para la prueba

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [4]:
spark = SparkSession.builder.appName("PruebaTecnica").getOrCreate()

# Carga de los datos

Información de las empresas

In [5]:
companies = spark.read.csv("SostenibilidadEmpresarial/Empresas_Sostenibilidad.csv", header=True, inferSchema=True)

In [6]:
companies.printSchema()

root
 |-- empresa_id: string (nullable = true)
 |-- nombre: string (nullable = true)
 |-- sector: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- consumo_energia: double (nullable = true)
 |-- emisiones_co2: double (nullable = true)
 |-- certificacion_iso14001: integer (nullable = true)



In [7]:
companies.head(5)

[Row(empresa_id='EMP000000', nombre='Empresa_0', sector='Energía', pais='Canadá', consumo_energia=44404.7, emisiones_co2=3217.3, certificacion_iso14001=0),
 Row(empresa_id='EMP000001', nombre='Empresa_1', sector='Finanzas', pais='China', consumo_energia=25576.11, emisiones_co2=16135.7, certificacion_iso14001=1),
 Row(empresa_id='EMP000002', nombre='Empresa_2', sector='Salud', pais='Japón', consumo_energia=29305.07, emisiones_co2=2224.93, certificacion_iso14001=0),
 Row(empresa_id='EMP000003', nombre='Empresa_3', sector='Transporte', pais='China', consumo_energia=44600.25, emisiones_co2=3764.32, certificacion_iso14001=1),
 Row(empresa_id='EMP000004', nombre='Empresa_4', sector='Tecnología', pais='Francia', consumo_energia=46039.63, emisiones_co2=11840.17, certificacion_iso14001=1)]

In [8]:
companies.count()

24627

`Empresas_Sostenibilidad.csv` contiene información de 24627 empresas, comprendiendo su identificador, nombre, sector, país, consumo de energía, emisiones y si está certicada en la norma ISO 14001 (1: si, 0: no).

Proyectos

In [9]:
projects = spark.read.csv("SostenibilidadEmpresarial/Proyectos_Energeticos.csv", header=True, inferSchema=True)

In [10]:
projects.printSchema()

root
 |-- proyecto_id: string (nullable = true)
 |-- empresa_id: string (nullable = true)
 |-- tipo_energia: string (nullable = true)
 |-- capacidad_generacion: double (nullable = true)
 |-- reduccion_emisiones: double (nullable = true)
 |-- costo_proyecto: double (nullable = true)
 |-- estado_proyecto: string (nullable = true)



In [11]:
projects.head(5)

[Row(proyecto_id='PROJ000000', empresa_id='EMP016520', tipo_energia='Solar', capacidad_generacion=9131.06, reduccion_emisiones=2270.88, costo_proyecto=26.03, estado_proyecto='Activo'),
 Row(proyecto_id='PROJ000001', empresa_id='EMP012923', tipo_energia='Solar', capacidad_generacion=2164.32, reduccion_emisiones=4886.1, costo_proyecto=364.36, estado_proyecto='Finalizado'),
 Row(proyecto_id='PROJ000002', empresa_id='EMP023136', tipo_energia='Biomasa', capacidad_generacion=2374.24, reduccion_emisiones=446.3, costo_proyecto=190.17, estado_proyecto='En desarrollo'),
 Row(proyecto_id='PROJ000003', empresa_id='EMP004436', tipo_energia='Geotérmica', capacidad_generacion=4065.11, reduccion_emisiones=4465.83, costo_proyecto=126.6, estado_proyecto='Finalizado'),
 Row(proyecto_id='PROJ000004', empresa_id='EMP021681', tipo_energia='Eólica', capacidad_generacion=4581.51, reduccion_emisiones=3280.46, costo_proyecto=227.13, estado_proyecto='Activo')]

In [12]:
projects.count()

25808

In [13]:
projects.select("estado_proyecto").distinct().show()

+---------------+
|estado_proyecto|
+---------------+
|     Finalizado|
|  En desarrollo|
|         Activo|
+---------------+



`Proyectos_Energeticos.csv` contiene información de 25808 projectos: identificador del proyecto, identificador de la empresa (clave foránea a la tabla de empresas), tipo de energía, capacidad de generación, reducción de emisiones, costo del proyecto y su estado (que puede ser "Activo"/"En desarrollo"/"Finalizado").

Regulaciones

In [14]:
regulations = spark.read.csv("SostenibilidadEmpresarial/Regulaciones_Ambientales.csv", header=True, inferSchema=True)

In [15]:
regulations.printSchema()

root
 |-- regulacion_id: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- limite_emisiones: double (nullable = true)
 |-- subsidios_renovables: integer (nullable = true)
 |-- impuesto_carbono: double (nullable = true)



In [16]:
regulations.head(5)

[Row(regulacion_id='REG000000', pais='India', limite_emisiones=3012.63, subsidios_renovables=1, impuesto_carbono=19.54),
 Row(regulacion_id='REG000001', pais='EEUU', limite_emisiones=7325.54, subsidios_renovables=1, impuesto_carbono=42.41),
 Row(regulacion_id='REG000002', pais='Alemania', limite_emisiones=9060.84, subsidios_renovables=1, impuesto_carbono=6.15),
 Row(regulacion_id='REG000003', pais='España', limite_emisiones=7075.95, subsidios_renovables=1, impuesto_carbono=44.65),
 Row(regulacion_id='REG000004', pais='Alemania', limite_emisiones=1997.42, subsidios_renovables=0, impuesto_carbono=8.75)]

In [17]:
regulations.count()

29707

`Regulaciones_Ambientales.csv` contiene información de 29707 regulaciones: identificador de la regulación, el país (por el cual se puede relacionar cada empresa), límite de emisiones, subsidios renovables (1: si, 0: no) y el impuesto al carbono.

# Validación de los datos

## Comprobación de duplicados

### Para cada conjunto de datos se comprueba si hay filas repetidas

In [18]:
def duplicates_all_columns(df):
  df.groupBy(df.columns) \
    .count() \
    .filter(F.col("count") > 1) \
    .show()

In [19]:
duplicates_all_columns(companies)

+----------+------+------+----+---------------+-------------+----------------------+-----+
|empresa_id|nombre|sector|pais|consumo_energia|emisiones_co2|certificacion_iso14001|count|
+----------+------+------+----+---------------+-------------+----------------------+-----+
+----------+------+------+----+---------------+-------------+----------------------+-----+



In [20]:
duplicates_all_columns(projects)

+-----------+----------+------------+--------------------+-------------------+--------------+---------------+-----+
|proyecto_id|empresa_id|tipo_energia|capacidad_generacion|reduccion_emisiones|costo_proyecto|estado_proyecto|count|
+-----------+----------+------------+--------------------+-------------------+--------------+---------------+-----+
+-----------+----------+------------+--------------------+-------------------+--------------+---------------+-----+



In [21]:
duplicates_all_columns(regulations)

+-------------+----+----------------+--------------------+----------------+-----+
|regulacion_id|pais|limite_emisiones|subsidios_renovables|impuesto_carbono|count|
+-------------+----+----------------+--------------------+----------------+-----+
+-------------+----+----------------+--------------------+----------------+-----+



No se detectan filas repetidas en el dataset.

### Se comprueba si hay claves repetidas en cada tabla

In [22]:
def duplicates_by_column(df, column):
  df.groupBy(column) \
    .count() \
    .filter(F.col("count") > 1) \
    .show()

In [23]:
duplicates_by_column(companies, "empresa_id")

+----------+-----+
|empresa_id|count|
+----------+-----+
+----------+-----+



In [24]:
duplicates_by_column(projects, "proyecto_id")

+-----------+-----+
|proyecto_id|count|
+-----------+-----+
+-----------+-----+



In [25]:
duplicates_by_column(regulations, "regulacion_id")

+-------------+-----+
|regulacion_id|count|
+-------------+-----+
+-------------+-----+



No aparecen claves repetidas. Se comprueba también que no haya más de una empresa con el mismo nombre en `companies`.

In [26]:
duplicates_by_column(companies, "nombre")

+------+-----+
|nombre|count|
+------+-----+
+------+-----+



### Se comprueba la existencia de valores nulos o vacíos

Para cada columna se cuenta el número de nulos en el caso de valores numéricos y nulos/cadenas vacías en el caso de *strings*

In [27]:
def check_nulls_or_empty_str(df):
  df.select([
      (
          F.sum(F.col(c).isNull().cast("int"))
          if dict(df.dtypes)[c] in ("int", "double", "float")
          else F.sum((F.col(c).isNull() | (F.col(c) == "")).cast("int"))
      ).alias(c)
      for c in df.columns
  ]).show()

In [28]:
check_nulls_or_empty_str(companies)


+----------+------+------+----+---------------+-------------+----------------------+
|empresa_id|nombre|sector|pais|consumo_energia|emisiones_co2|certificacion_iso14001|
+----------+------+------+----+---------------+-------------+----------------------+
|         0|     0|     0|   0|              0|            0|                     0|
+----------+------+------+----+---------------+-------------+----------------------+



In [29]:
check_nulls_or_empty_str(projects)

+-----------+----------+------------+--------------------+-------------------+--------------+---------------+
|proyecto_id|empresa_id|tipo_energia|capacidad_generacion|reduccion_emisiones|costo_proyecto|estado_proyecto|
+-----------+----------+------------+--------------------+-------------------+--------------+---------------+
|          0|         0|           0|                   0|                  0|             0|              0|
+-----------+----------+------------+--------------------+-------------------+--------------+---------------+



In [30]:
check_nulls_or_empty_str(regulations)

+-------------+----+----------------+--------------------+----------------+
|regulacion_id|pais|limite_emisiones|subsidios_renovables|impuesto_carbono|
+-------------+----+----------------+--------------------+----------------+
|            0|   0|               0|                   0|               0|
+-------------+----+----------------+--------------------+----------------+



Todos los datos parecen estar correctamente formados dentro de cada tabla.

### A continuación se comprueba la integridad referencial

Se busca inconsistencia en la tabla de proyectos en sus referencias a la tabla de empresas. Se comprueba si hay algún proyecto con un identificador de empresa inexistente.

In [31]:
companies_keys = companies.select("empresa_id").dropDuplicates()

projects_unknown_company = projects.join(
    companies_keys,
    on="empresa_id",
    how="left_anti"
)

print(projects_unknown_company.count())

0


Todos los proyectos apuntan a empresas existentes en `companies`.

### Aunque no afecte a las operaciones, se realiza la operación inversa para contar las empresas que no cuenten con proyectos energéticos.

In [32]:
projects_keys = projects.select("empresa_id").dropDuplicates()

companies_without_projects = companies.join(
    projects_keys,
    on="empresa_id",
    how="left_anti"
)

print(companies_without_projects.count())
print(companies_without_projects.show(5))

8594
+----------+----------+-----------+------+---------------+-------------+----------------------+
|empresa_id|    nombre|     sector|  pais|consumo_energia|emisiones_co2|certificacion_iso14001|
+----------+----------+-----------+------+---------------+-------------+----------------------+
| EMP000002| Empresa_2|      Salud| Japón|       29305.07|      2224.93|                     0|
| EMP000003| Empresa_3| Transporte| China|       44600.25|      3764.32|                     1|
| EMP000006| Empresa_6|    Energía| China|        2793.22|      3871.34|                     0|
| EMP000009| Empresa_9|    Energía| China|        6868.81|      13398.5|                     1|
| EMP000012|Empresa_12|Manufactura|Canadá|       24567.02|      1807.39|                     0|
+----------+----------+-----------+------+---------------+-------------+----------------------+
only showing top 5 rows

None


Hay 8594 empresas que no cuentan con proyectos energéticos.

### Ahora se realiza la misma comprobación para la relación entre empresas y regulaciones por país.

In [33]:
companies_countries = companies.select("pais").dropDuplicates()

regulations_unknown_country = regulations.join(
    companies_countries,
    on="pais",
    how="left_anti"
)
regulations_unknown_country.count()

0

In [34]:
regulations_countries = regulations.select("pais").dropDuplicates()

companies_unknown_country = companies.join(
    regulations_countries,
    on="pais",
    how="left_anti"
)
companies_unknown_country.count()

0

Todas las empresas y regulaciones muestran países que se encuentran en ambas tablas.

# Procesamiento de los datos

## Creación de la tabla de `Impacto Ambiental Empresas`

Se cargan de nuevo los datos para no depender de las ejecuciones anteriores.

In [35]:
companies = spark.read.csv("SostenibilidadEmpresarial/Empresas_Sostenibilidad.csv", header=True, inferSchema=True)
projects = spark.read.csv("SostenibilidadEmpresarial/Proyectos_Energeticos.csv", header=True, inferSchema=True)
regulations = spark.read.csv("SostenibilidadEmpresarial/Regulaciones_Ambientales.csv", header=True, inferSchema=True)

Se juntan los datos de empresas y proyectos.

In [36]:
companies_and_projects = companies.join(
    projects,
    on="empresa_id",
    how="left"
)

companies_and_projects.show(5)

+----------+---------+----------+------+---------------+-------------+----------------------+-----------+--------------+--------------------+-------------------+--------------+---------------+
|empresa_id|   nombre|    sector|  pais|consumo_energia|emisiones_co2|certificacion_iso14001|proyecto_id|  tipo_energia|capacidad_generacion|reduccion_emisiones|costo_proyecto|estado_proyecto|
+----------+---------+----------+------+---------------+-------------+----------------------+-----------+--------------+--------------------+-------------------+--------------+---------------+
| EMP000000|Empresa_0|   Energía|Canadá|        44404.7|       3217.3|                     0| PROJ020806|        Eólica|             1962.66|             606.28|        375.11|  En desarrollo|
| EMP000000|Empresa_0|   Energía|Canadá|        44404.7|       3217.3|                     0| PROJ002488|Hidroeléctrica|             9150.33|             224.96|        301.21|         Activo|
| EMP000001|Empresa_1|  Finanzas| C

Se calcula el total de reducción de emisiones por empresa sumando la reducción de emisiones de todos los proyectos de cada empresa. Después, se resta las emisiones de la empresa menos el total de reducción de emisiones.

In [37]:
companies_net_emissions = companies_and_projects.groupBy("empresa_id").agg(
    F.first("nombre").alias("nombre"),
    F.first("pais").alias("pais"),
    F.first("emisiones_co2").alias("emisiones_co2"),
    F.round(F.sum("reduccion_emisiones"), 2).alias("total_reduccion_emisiones")
)

# Se substituyen nulls por 0
companies_net_emissions = companies_net_emissions.fillna({"total_reduccion_emisiones": 0.0})

companies_net_emissions = companies_net_emissions.withColumn(
    "emisiones_netas",
    F.round(F.col("emisiones_co2") - F.col("total_reduccion_emisiones"), 2)
)

companies_net_emissions.show(5)

+----------+---------+-------+-------------+-------------------------+---------------+
|empresa_id|   nombre|   pais|emisiones_co2|total_reduccion_emisiones|emisiones_netas|
+----------+---------+-------+-------------+-------------------------+---------------+
| EMP000000|Empresa_0| Canadá|       3217.3|                   831.24|        2386.06|
| EMP000001|Empresa_1|  China|      16135.7|                  1565.71|       14569.99|
| EMP000002|Empresa_2|  Japón|      2224.93|                      0.0|        2224.93|
| EMP000003|Empresa_3|  China|      3764.32|                      0.0|        3764.32|
| EMP000004|Empresa_4|Francia|     11840.17|                  3963.83|        7876.34|
+----------+---------+-------+-------------+-------------------------+---------------+
only showing top 5 rows



Dado que en la información sobre regulaciones, cada país aparece repetido múltiples veces, se debe elegir una regulación única por país. Se podría tener en cuenta para la decisión el impuesto, pero el criterio más razonable parece el de seleccionar la regulación más restrictiva por país, es decir, la que cuente con el menor límite de emisiones.

In [38]:
w = Window.partitionBy("pais").orderBy(F.col("limite_emisiones").asc())

regulations_most_restrictive = (
    regulations
    .withColumn("row_index", F.row_number().over(w))  # añadimos un índice en cada país
    .filter(F.col("row_index") == 1)
    .drop("row_index")
)
regulations_most_restrictive.show(5)

+-------------+--------+----------------+--------------------+----------------+
|regulacion_id|    pais|limite_emisiones|subsidios_renovables|impuesto_carbono|
+-------------+--------+----------------+--------------------+----------------+
|    REG007043|Alemania|         1000.79|                   1|           24.27|
|    REG004617|  Brasil|         1001.14|                   0|           14.45|
|    REG026381|  Canadá|         1001.43|                   1|           35.78|
|    REG006127|   China|         1004.49|                   1|           16.83|
|    REG024888|    EEUU|         1000.51|                   1|           41.07|
+-------------+--------+----------------+--------------------+----------------+
only showing top 5 rows



Se asigna a cada empresa la regulación correspondiente y se eliminan columnas que no son necesarias para esta tarea.

In [39]:
companies_and_regulations = companies_net_emissions.join(
    regulations_most_restrictive,
    on="pais",
    how="left"
).drop("pais", "emisiones_co2", "total_reduccion_emisiones", "subsidios_renovables")

companies_and_regulations.show(5)

+----------+---------+---------------+-------------+----------------+----------------+
|empresa_id|   nombre|emisiones_netas|regulacion_id|limite_emisiones|impuesto_carbono|
+----------+---------+---------------+-------------+----------------+----------------+
| EMP000000|Empresa_0|        2386.06|    REG026381|         1001.43|           35.78|
| EMP000001|Empresa_1|       14569.99|    REG006127|         1004.49|           16.83|
| EMP000002|Empresa_2|        2224.93|    REG009011|         1000.25|           13.23|
| EMP000003|Empresa_3|        3764.32|    REG006127|         1004.49|           16.83|
| EMP000004|Empresa_4|        7876.34|    REG029620|         1002.32|           21.76|
+----------+---------+---------------+-------------+----------------+----------------+
only showing top 5 rows



Se crea una columna nueva conforme las emisiones netas están o no por debajo del límite de la regulación correspondiente. También se añade una columna de forma temporal con el exceso de emisiones para facilitar el cálculo posterior.

In [40]:
companies_and_regulations = companies_and_regulations.withColumn(
    "complies_with_regulation",
    F.col("emisiones_netas") < F.col("limite_emisiones")
).withColumn(
    "exceso_emisiones",
    F.when(
        ~F.col("complies_with_regulation"),
        F.round(F.col("emisiones_netas") - F.col("limite_emisiones"), 2)
    ).otherwise(F.lit(0))
)

companies_and_regulations.show(20)

+----------+----------+---------------+-------------+----------------+----------------+------------------------+----------------+
|empresa_id|    nombre|emisiones_netas|regulacion_id|limite_emisiones|impuesto_carbono|complies_with_regulation|exceso_emisiones|
+----------+----------+---------------+-------------+----------------+----------------+------------------------+----------------+
| EMP000000| Empresa_0|        2386.06|    REG026381|         1001.43|           35.78|                   false|         1384.63|
| EMP000001| Empresa_1|       14569.99|    REG006127|         1004.49|           16.83|                   false|         13565.5|
| EMP000002| Empresa_2|        2224.93|    REG009011|         1000.25|           13.23|                   false|         1224.68|
| EMP000003| Empresa_3|        3764.32|    REG006127|         1004.49|           16.83|                   false|         2759.83|
| EMP000004| Empresa_4|        7876.34|    REG029620|         1002.32|           21.76|   

Entendiendo el `impuesto_carbono` como un impuesto aplicado por unidad de emisiones de co2, se calcula el impuesto para cada empresa en base al exceso sobre el límite de la regulación correspondiente.

In [41]:
impacto_ambiental_empresas = companies_and_regulations.withColumn(
    "costo_impuesto",
    F.round(F.col("exceso_emisiones") * F.col("impuesto_carbono"), 2)
).drop("limite_emisiones", "impuesto_carbono", "exceso_emisiones")

impacto_ambiental_empresas.show(20)

+----------+----------+---------------+-------------+------------------------+--------------+
|empresa_id|    nombre|emisiones_netas|regulacion_id|complies_with_regulation|costo_impuesto|
+----------+----------+---------------+-------------+------------------------+--------------+
| EMP000000| Empresa_0|        2386.06|    REG026381|                   false|      49542.06|
| EMP000001| Empresa_1|       14569.99|    REG006127|                   false|     228307.37|
| EMP000002| Empresa_2|        2224.93|    REG009011|                   false|      16202.52|
| EMP000003| Empresa_3|        3764.32|    REG006127|                   false|      46447.94|
| EMP000004| Empresa_4|        7876.34|    REG029620|                   false|     149578.68|
| EMP000005| Empresa_5|        2147.87|    REG026381|                   false|      41019.62|
| EMP000006| Empresa_6|        3871.34|    REG006127|                   false|      48249.09|
| EMP000007| Empresa_7|        6224.86|    REG004617|       

Por consistencia, se muestra el valor booleano de `cumple_normativa` usando 0 y 1.

In [42]:
impacto_ambiental_empresas = impacto_ambiental_empresas.withColumn(
    "cumple_normativa",
    F.col("complies_with_regulation").cast("int")
).drop("complies_with_regulation")

impacto_ambiental_empresas.show(20)

+----------+----------+---------------+-------------+--------------+----------------+
|empresa_id|    nombre|emisiones_netas|regulacion_id|costo_impuesto|cumple_normativa|
+----------+----------+---------------+-------------+--------------+----------------+
| EMP000000| Empresa_0|        2386.06|    REG026381|      49542.06|               0|
| EMP000001| Empresa_1|       14569.99|    REG006127|     228307.37|               0|
| EMP000002| Empresa_2|        2224.93|    REG009011|      16202.52|               0|
| EMP000003| Empresa_3|        3764.32|    REG006127|      46447.94|               0|
| EMP000004| Empresa_4|        7876.34|    REG029620|     149578.68|               0|
| EMP000005| Empresa_5|        2147.87|    REG026381|      41019.62|               0|
| EMP000006| Empresa_6|        3871.34|    REG006127|      48249.09|               0|
| EMP000007| Empresa_7|        6224.86|    REG004617|      75482.75|               0|
| EMP000008| Empresa_8|       10384.66|    REG008575| 

Se realiza una comprobación rápida de que el número de empresas resultantes es el mismo que en el dataset inicial.

In [43]:
companies.count() == impacto_ambiental_empresas.count()

True

## Creación de la tabla de `Beneficios Proyectos Energéticos`

In [44]:
projects = spark.read.csv("SostenibilidadEmpresarial/Proyectos_Energeticos.csv", header=True, inferSchema=True)

Por la lógica de negocio, la tabla resultante tendrá 1 fila por empresa y tipo de energía. Para cada uno de estos conjuntos se suma el costo de los proyectos, la capacidad de generación y las emisiones evitadas. Finalmente se calcula el retorno estimado dividiendo la inversión total entre el total de emisiones evitadas.

In [45]:
beneficios_proyectos_energeticos = projects.groupBy("empresa_id", "tipo_energia").agg(
    F.round(F.sum("costo_proyecto"), 2).alias("total_inversion"),
    F.round(F.sum("capacidad_generacion"), 2).alias("energia_generada_total"),
    F.round(F.sum("reduccion_emisiones"), 2).alias("emisiones_evitadas_total")
).withColumn(
    "retorno_estimado",
    F.round(F.col("total_inversion") / F.col("emisiones_evitadas_total"), 4)
).orderBy("empresa_id", "tipo_energia")

beneficios_proyectos_energeticos.show(10)

+----------+--------------+---------------+----------------------+------------------------+----------------+
|empresa_id|  tipo_energia|total_inversion|energia_generada_total|emisiones_evitadas_total|retorno_estimado|
+----------+--------------+---------------+----------------------+------------------------+----------------+
| EMP000000|        Eólica|         375.11|               1962.66|                  606.28|          0.6187|
| EMP000000|Hidroeléctrica|         301.21|               9150.33|                  224.96|          1.3389|
| EMP000001|       Biomasa|           4.58|               7295.54|                 1565.71|          0.0029|
| EMP000004|       Biomasa|         249.27|                948.21|                 3963.83|          0.0629|
| EMP000005|    Geotérmica|         151.59|               3851.69|                 1551.77|          0.0977|
| EMP000005|         Solar|         130.73|                5587.7|                  1965.9|          0.0665|
| EMP000007|       

In [46]:
beneficios_proyectos_energeticos.count()

23218

# Escritura de los datos

In [47]:
#!rm -r output

In [48]:
impacto_ambiental_empresas.write.option("header", True).mode("overwrite").csv("output/impacto_ambiental_empresas")
beneficios_proyectos_energeticos.write.option("header", True).mode("overwrite").csv("output/beneficios_proyectos_energeticos")